<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/StockFractionRobustness1/61_SAC_Fraksi1_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data saham-saham fraksi 2 dijalankan menggunakan email shintaroudlotulhanafia@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



**Memasangkan Google Drive ke Google Colab**

* Kata kunci **from** digunakan untuk mengimpor hanya bagian tertentu dari modul (melakukan impor untuk bagian tertentu pada *library*)
* Modul merupakan file yang berisi sekumpulan fungsi yang ingin disertakan dalam aplikasi. Untuk membuat modul cukup simpan kode yang Anda inginkan dalam file dengan ekstensi file .py.
* **import** melakukan impor seluruh *library*.
* **Mounting** adalah proses membuat file dan direktori pada perangkat penyimpanan (seperti google drive) sehingga tersedia bagi pengguna untuk dapat diakses.



# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
    * [7.4. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi (dalam bentuk string)](#6.3)
      * [7.4.1. Agen 1: A2C](#6.4.1)
      * [7.4.2. Agen 2: PPO](#6.4.2)
      * [7.4.3. Agen 3: TD3](#6.4.3)
      * [7.4.4. Agen 4: SAC](#6.4.4)
      * [7.4.5. Agen 5: DDPG](#6.4.5)
    * [7.5. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Kondisi dan Aksi](#6.4)
      * [7.5.1. Agen 1: A2C](#6.4.1)
      * [7.5.2. Agen 2: PPO](#6.4.2)
      * [7.5.3. Agen 3: TD3](#6.4.3)
      * [7.5.4. Agen 4: SAC](#6.4.4)
      * [7.5.5. Agen 5: DDPG](#6.4.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)
    * [8.2. Gambaran BackTesting](#7.2)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


* FinRL adalah *open-source framework* pertama yang menunjukkan potensi besar *Reinforcement Learning* keuangan.

* FinRL menyediakan berbagai pengaturan untuk melakukan *trading* dengan *Reinforcement Learning* seperti menyediakan ratusan pasar keuangan, algoritma yang canggih, berbagai macam aplikasi keuangan (alokasi portofolio, perdagangan mata uang kripto, *high-frequency trading*), *live trading, cloud deployment,* dll.

* Pengaplikasian pustaka FinRL dapat menggunakan alamat berikut, git+https://github.com/AI4Finance-Foundation/FinRL.git. Namun, untuk kemudahan pengaturan nilai sesuai kebutuhan, maka, dilakukanlah *forking* terhadap *repository* tersebut, lalu digunakanlah alamat berikut, git+https://github.com/shintaroudlotulhanafia/FinRL.git



In [2]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-xc14u_f7
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-xc14u_f7
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-wqombtc6/pyfolio_5a5f80c748b24e7494398c32ad5d8571
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-wqombtc6/pyfolio_5a5f80c748b24e7494398c32ad5d8571
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-wqombtc6/elegantrl_b409133109ae4f998eef5197c6ec22cd
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-wqombtc6/elegantrl_b409133109ae4f998eef5197c6ec22cd
     |████████████████████████████████| 2.3 MB 8.4 MB/s 
     |████████████████████████████████| 234 kB 79.2 MB/s 
     |██████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

* **config** berisi pengaturan tanggal periode training dan trading, indikator, hyperparameter setiap model/agen DRL.
* **config_tickers** berisi pengaturan daftar saham yang akan diproses.
* **import os** digunakan untuk meng-import modul, merupakan module pada python agar python itu sendiri berinteraksi langsung terhadap sistem operasi.
* **check_and_make_directories** digunakan untuk mengecek dan membuat folder.


In [4]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

-----
kelas YahooDownloader:
    Mengambil data saham harian dari
    Yahoo Finance API

    Atribut
    ----------
        start_date : str
            tanggal mulai data
        end_date : str
            tanggal akhir data
        ticker_list : list
            daftar ticker saham (dimodifikasi dari config.py)

    Methods
    -------
    fetch_data()


Penetapan tanggal training dan trading dapat dilakukan dengan mengaturnya pada finrl/config.py atau dengan menetapkannya di sel notebook.

In [5]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [6]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: <Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00 - Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00 - Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00 - Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']



In [7]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi1).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10916, 8)


* **df.shape** digunakan untuk mendapatkan jumlah baris dan kolom

In [8]:
df.shape

(10916, 8)

* **nunique()** mengembalikan jumlah nilai unik untuk setiap kolom. Dengan menentukan sumbu kolom ( axis='columns' ), metode nunique() mencari berdasarkan kolom tersebut dan mengembalikan jumlah nilai unik untuk setiap baris.

In [9]:
df.nunique()

date      2729
open      1014
high      1044
low       1021
close     1441
volume    8684
tic          4
day          5
dtype: int64

* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [10]:
print(df['tic'].unique())

['KIJA.JK' 'LCGP.JK' 'LMPI.JK' 'LPKR.JK']


* **describe()** digunakan untuk menampilkan detail statistik dasar seperti persentil, mean, std, dll. Hanya kolom yang bertipe numerik yang akan ditampilkan statistiknya.

In [11]:
df.describe()

,open,high,low,close,volume,day
count,10916.000000,10916.000000,10916.000000,10916.000000,1.091600e+04,10916.000000
mean,305.131510,310.550823,299.173490,299.052477,4.325543e+07,1.988641
std,238.405992,242.413365,233.786952,228.963013,7.793921e+07,1.409951
min,48.857891,48.857891,48.857891,48.236725,0.000000e+00,0.000000
25%,131.000000,135.000000,128.007675,130.000000,1.500000e+05,1.000000
50%,235.000000,240.000000,230.000000,235.000000,1.444527e+07,2.000000
75%,395.196686,400.751678,384.761902,379.425934,5.852571e+07,3.000000
max,1460.164551,1468.100220,1412.550415,1382.835327,1.637779e+09,4.000000


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan *ascending* atau *descending* dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [12]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,48.857891,49.835049,48.857891,48.236725,96630243,KIJA.JK,0
1,2009-01-05,50.000000,50.000000,50.000000,50.000000,0,LCGP.JK,0
2,2009-01-05,70.000000,70.000000,70.000000,70.000000,0,LMPI.JK,0
3,2009-01-05,596.003723,641.269836,596.003723,581.263184,32545542,LPKR.JK,0
4,2009-01-06,48.857891,52.766521,48.857891,48.236725,226697765,KIJA.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.



-----
kelas FeatureEngineer:
    Menyediakan metode untuk preprocessing data harga saham

    Attributes
    ----------
        use_technical_indicator : boolean
            menggunakan indikator teknis atau tidak
        tech_indicator_list : list
            daftar nama indikator teknis (dimodifikasi dari neofinrl_config.py)
        use_vix : boolean
            menggunakan Volatility Index (VIX) atau tidak
        use_turbulence : boolean
            menggunakan indeks turbulensi atau tidak
        user_defined_feature:boolean
            menggunakan fitur yang ditentukan pengguna atau tidak
    Methods
    -------
    preprocess_data()
        metode utama untuk melakukan feature engineering


In [13]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [14]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [15]:
processed.describe()

,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
count,10608.000000,10608.000000,10608.000000,10608.000000,1.060800e+04,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000,10608.000000
mean,305.551516,310.995315,299.558443,299.458118,4.353285e+07,2.019231,0.054662,50.356373,3.225394,18.179936,4.082293
std,238.745324,242.759425,234.098488,229.299717,7.774323e+07,1.397923,10.822138,9.970338,125.018908,7.335005,11.896582
min,48.857891,48.857891,48.857891,48.236725,0.000000e+00,0.000000,-80.906874,21.123967,-1000.000000,9.140000,0.000000
25%,131.000000,134.961945,128.005756,130.000000,1.550000e+05,1.000000,-2.768206,44.182374,-81.245432,13.287500,0.415048
50%,235.000000,240.000000,230.000000,235.000000,1.468182e+07,2.000000,-0.000153,49.056784,-10.579830,15.985000,1.401532
75%,396.783813,405.130127,384.761902,381.000305,5.901394e+07,3.000000,3.141874,55.150525,76.738157,20.629999,3.692576
max,1460.164551,1468.100220,1412.550415,1382.835327,1.637779e+09,4.000000,80.156574,100.000000,1000.000000,56.650002,364.269000


In [16]:
processed.nunique()

date           2652
open           1010
high           1038
low            1015
close          1434
volume         8472
tic               4
day               5
macd          10357
rsi_30         7482
cci_30         9825
vix            1441
turbulence     2399
dtype: int64

* **tolist()** digunakan untuk mengubah elemen data array menjadi *list*.
* **unique()** digunakan untuk menemukan elemen unik dari array. Mengembalikan elemen unik yang diurutkan dari sebuah array.

In [17]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

* **date_range()** digunakan untuk mendapatkan frekuensi tetap DatetimeIndex. date_range() digunakan untuk membuat rentang tanggal di pandas.
* **max()** mengembalikan item dengan nilai tertinggi, atau item dengan nilai tertinggi dalam *iterable*. Jika nilainya adalah string, maka dilakukan perbandingan abjad.
* **min()** mengembalikan item dengan nilai terendah, atau item dengan nilai terendah dalam iterable. Jika nilainya adalah string,  maka dilakukan perbandingan abjad.
* **astype()** digunakan untuk mengubah tipe data dari suatu bentuk *series*.

In [18]:
list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

* **Array** dapat menyimpan elemen hanya dari satu tipe. Sedangkan **list** juga dapat menyimpan elemen dari tipe data yang berbeda.
* **list()** digunakan untuk membuat objek *list*. Objek *list* adalah kumpulan yang berurutan dan dapat diubah.
* **itertools.product()** digunakan untuk mencari produk kartesius dari iterator yang diberikan, outputnya adalah urutan leksikografis.

In [19]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'KIJA.JK'),
 ('2009-01-05', 'LCGP.JK'),
 ('2009-01-05', 'LMPI.JK'),
 ('2009-01-05', 'LPKR.JK'),
 ('2009-01-06', 'KIJA.JK'),
 ('2009-01-06', 'LCGP.JK'),
 ('2009-01-06', 'LMPI.JK'),
 ('2009-01-06', 'LPKR.JK'),
 ('2009-01-07', 'KIJA.JK'),
 ('2009-01-07', 'LCGP.JK'),
 ('2009-01-07', 'LMPI.JK'),
 ('2009-01-07', 'LPKR.JK'),
 ('2009-01-08', 'KIJA.JK'),
 ('2009-01-08', 'LCGP.JK'),
 ('2009-01-08', 'LMPI.JK'),
 ('2009-01-08', 'LPKR.JK'),
 ('2009-01-09', 'KIJA.JK'),
 ('2009-01-09', 'LCGP.JK'),
 ('2009-01-09', 'LMPI.JK'),
 ('2009-01-09', 'LPKR.JK'),
 ('2009-01-10', 'KIJA.JK'),
 ('2009-01-10', 'LCGP.JK'),
 ('2009-01-10', 'LMPI.JK'),
 ('2009-01-10', 'LPKR.JK'),
 ('2009-01-11', 'KIJA.JK'),
 ('2009-01-11', 'LCGP.JK'),
 ('2009-01-11', 'LMPI.JK'),
 ('2009-01-11', 'LPKR.JK'),
 ('2009-01-12', 'KIJA.JK'),
 ('2009-01-12', 'LCGP.JK'),
 ('2009-01-12', 'LMPI.JK'),
 ('2009-01-12', 'LPKR.JK'),
 ('2009-01-13', 'KIJA.JK'),
 ('2009-01-13', 'LCGP.JK'),
 ('2009-01-13', 'LMPI.JK'),
 ('2009-01-13', 'LPK

* **Pandas DataFrame** adalah dua dimensi yang dapat berubah ukuran, struktur data tabular yang berpotensi heterogen dengan sumbu berlabel (baris dan kolom).
* **merge()** berfungsi untuk memperbarui konten dua DataFrame dengan menggabungkannya bersama-sama, menggunakan metode yang ditentukan.

In [20]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15999,2019-12-18,LPKR.JK,238.000000,240.000000,236.000000,238.000000,57097600.0,2.0,-1.505139,45.323461,-96.247961,12.580000,0.752572
16000,2019-12-19,KIJA.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16001,2019-12-19,LCGP.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16002,2019-12-19,LMPI.JK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* **isin()** digunakan untuk memfilter data frames. isin() dapat membantu untuk memilih baris dengan memiliki nilai tertentu (atau beberapa nilai tertentu) dalam kolom tertentu.

In [21]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


In [22]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


* **fillna()** digunakan untuk mengisi nilai NA/NaN menggunakan metode yang ditentukan. fillna(0) artinya baris NA/NaN diisi dengan nilai misalnya 0.

In [23]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,240.000000,244.000000,238.000000,240.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,298.000000,306.000000,292.000000,296.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,114.000000,114.000000,114.000000,114.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,101.000000,101.000000,101.000000,101.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


In [24]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,48.857891,49.835049,48.857891,48.236725,96630243.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
1,2009-01-05,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
2,2009-01-05,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
3,2009-01-05,LPKR.JK,596.003723,641.269836,596.003723,581.263184,32545542.0,0.0,0.000000,100.0,66.666667,39.080002,0.0
4,2009-01-06,KIJA.JK,48.857891,52.766521,48.857891,48.236725,226697765.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
5,2009-01-06,LCGP.JK,50.000000,50.000000,50.000000,50.000000,0.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
6,2009-01-06,LMPI.JK,70.000000,70.000000,70.000000,70.000000,0.0,1.0,0.000000,100.0,66.666667,38.560001,0.0
7,2009-01-06,LPKR.JK,626.181152,648.814209,626.181152,602.272644,53837632.0,1.0,0.471366,100.0,66.666667,38.560001,0.0
8,2009-01-07,KIJA.JK,48.857891,51.789364,48.857891,48.236725,203203608.0,2.0,0.000000,100.0,20.000000,43.389999,0.0
9,2009-01-07,LCGP.JK,50.000000,50.000000,50.000000,50.000000,10000.0,2.0,0.000000,100.0,20.000000,43.389999,0.0


In [25]:
processed_full.nunique()

date           2652
tic               4
open           1010
high           1038
low            1015
close          1434
volume         8472
day               5
macd          10357
rsi_30         7482
cci_30         9825
vix            1441
turbulence     2399
dtype: int64

* **info()** berfungsi untuk mencetak informasi tentang DataFrame. Informasi tersebut berisi jumlah kolom, label kolom, tipe data kolom, penggunaan memori, indeks rentang, dan jumlah sel di setiap kolom (nilai bukan nol).

In [26]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10608 entries, 0 to 15999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        10608 non-null  object 
 1   tic         10608 non-null  object 
 2   open        10608 non-null  float64
 3   high        10608 non-null  float64
 4   low         10608 non-null  float64
 5   close       10608 non-null  float64
 6   volume      10608 non-null  float64
 7   day         10608 non-null  float64
 8   macd        10608 non-null  float64
 9   rsi_30      10608 non-null  float64
 10  cci_30      10608 non-null  float64
 11  vix         10608 non-null  float64
 12  turbulence  10608 non-null  float64
dtypes: float64(11), object(2)
memory usage: 1.1+ MB


* **to_csv()** mengonversi DataFrame menjadi data CSV/comma separated value (nilai yang dipisahkan koma). Kita dapat melewatkan objek file untuk menulis data CSV ke dalam file. Jika tidak, data CSV dikembalikan dalam format string.
* **loc** berbasis label, yang berarti baris dan kolom diambil berdasarkan label baris dan kolomnya. **iloc** berbasis posisi integer, maka baris dan kolom dapat diambil dengan menggunakan nilai posisi integernya (posisi integer berbasis 0).

In [28]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,KIJA.JK,4885.789108,4983.504868,4885.789108,4823.672485,96630243.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
1,2009-01-05,LCGP.JK,5000.000000,5000.000000,5000.000000,5000.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
2,2009-01-05,LMPI.JK,7000.000000,7000.000000,7000.000000,7000.000000,0.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
3,2009-01-05,LPKR.JK,59600.372314,64126.983643,59600.372314,58126.318359,32545542.0,0.0,0.000000,100.000000,66.666667,39.080002,0.000000
4,2009-01-06,KIJA.JK,4885.789108,5276.652145,4885.789108,4823.672485,226697765.0,1.0,0.000000,100.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,LPKR.JK,24000.000000,24400.000000,23800.000000,24000.000000,84248500.0,1.0,-1.308380,46.675013,-57.447362,12.290000,0.008272
15996,2019-12-18,KIJA.JK,29800.000000,30600.000000,29200.000000,29600.000000,8984100.0,2.0,-5.423601,33.586043,-158.751696,12.580000,0.752572
15997,2019-12-18,LCGP.JK,11400.000000,11400.000000,11400.000000,11400.000000,0.0,2.0,-0.000004,44.937077,-158.751696,12.580000,0.752572
15998,2019-12-18,LMPI.JK,10100.000000,10100.000000,10100.000000,10100.000000,400.0,2.0,-4.910314,40.464511,-43.939546,12.580000,0.752572


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

* **data_split** membagi data menjadi himpunan *train, test,* dan/atau *validation.*
* **len()** mengembalikan jumlah item dalam suatu objek. Namun, jika objek adalah string, maka fungsi len() akan mengembalikan jumlah karakter dalam string.
* **tail()** digunakan untuk mendapatkan n baris terakhir.

In [29]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1980


In [30]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,LPKR.JK,38884.814453,39519.668579,38408.673096,38257.470703,58597272.0,3.0,-28.385327,27.868502,-102.555067,10.18,1.611104
2156,2017-12-29,KIJA.JK,28600.000000,28600.000000,28200.000000,28600.000000,16508800.0,4.0,-3.612968,34.278597,-97.415507,11.04,0.257610
2156,2017-12-29,LCGP.JK,7300.000000,8800.000000,7300.000000,8000.000000,8834300.0,4.0,-1.389227,43.985598,-8.855912,11.04,0.257610
2156,2017-12-29,LMPI.JK,16800.000000,17100.000000,16700.000000,16700.000000,35200.0,4.0,-1.008401,47.293358,-102.476931,11.04,0.257610
2156,2017-12-29,LPKR.JK,38884.814453,39202.243042,38567.388916,38414.910889,39843609.0,4.0,-27.095883,28.442145,-97.849420,11.04,0.257610


In [31]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363
0,2018-01-02,LCGP.JK,7900.000000,8300.000000,7900.000000,8100.000000,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363
0,2018-01-02,LMPI.JK,17100.000000,17200.000000,16700.000000,16800.000000,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363
0,2018-01-02,LPKR.JK,38884.814453,39043.527222,38408.673096,38100.030518,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363
1,2018-01-03,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529


In [32]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


Transaction Fee
    # BUY Fee => 0.23% (Broker Fee(0.19%) + Levy(0.04%))
    # SELL Fee => 0.54% (Broker Fee(0.29%) + Levy(0.04%) + PPN(0.11%) + PPh(0.1%))

In [33]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [34]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.



-----
kelas DRLAgent:
    Menyediakan implementasi untuk algoritma DRL

    Atribut
    ----------
        env: gym environment class
            kelas yang ditentukan pengguna

    Methods
    -------
        get_model()
            mengatur algoritma DRL
        train_model()
            melatih algoritma DRL dalam set data latih dan mengeluarkan model yang dilatih
        DRL_prediction()
            membuat prediksi dalam kumpulan data pengujian dan dapatkan hasil


In [35]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.4'></a>
##6.4. Agen 2: SAC

In [36]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [37]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 62       |
|    time_elapsed    | 137      |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | 1.62e+04 |
|    critic_loss     | 1.44e+06 |
|    ent_coef        | 1.25     |
|    learning_rate   | 0.0001   |
|    n_updates       | 8613     |
|    reward          | -78.11   |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 62       |
|    time_elapsed    | 275      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 9.04e+03 |
|    critic_loss     | 2.79e+05 |
|    ent_coef        | 1.25     |
|    learning_rate   | 0.0001   |
|    n_updates       | 17241    |
|    reward          | -78.11   |
---------------------------------
day: 2156, episode: 10
begin_total_asset: 100000

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp10.000.000.

Pengaturan ambang turbulensi *(turbulence threshold)*

Atur ambang turbulensi menjadi lebih besar dari maksimum dalam data turbulensi sampel. Jika indeks turbulensi saat ini lebih besar dari ambang batas, maka dapat diasumsikan bahwa pasar pada periode tersebut sedang bergejolak

* **drop_duplicates()** menghapus baris duplikat.
* **quantile()** digunakan untuk menghitung kuantil nilai dalam sumbu tertentu. Sumbu default adalah baris.

In [38]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [39]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [40]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [41]:
insample_risk_indicator.turbulence.describe()

count    2157.000000
mean        4.121638
std        12.797610
min         0.000000
25%         0.391273
50%         1.345584
75%         3.532035
max       364.269000
Name: turbulence, dtype: float64

In [42]:
insample_risk_indicator.turbulence.quantile(0.996)

79.1987096139747

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [43]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [44]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,8749200.0,1.0,-3.498002,32.441935,-113.728129,9.77,1.048363
0,2018-01-02,LCGP.JK,7900.000000,8300.000000,7900.000000,8100.000000,6255600.0,1.0,-1.088875,44.851317,2.335236,9.77,1.048363
0,2018-01-02,LMPI.JK,17100.000000,17200.000000,16700.000000,16800.000000,3400.0,1.0,-1.096968,48.158048,-62.500000,9.77,1.048363
0,2018-01-02,LPKR.JK,38884.814453,39043.527222,38408.673096,38100.030518,26017693.0,1.0,-24.700852,27.966124,-88.363095,9.77,1.048363
1,2018-01-03,KIJA.JK,28600.000000,28600.000000,28000.000000,28400.000000,15693500.0,2.0,-3.471229,32.441935,-111.111111,9.15,0.089529


<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

In [45]:
#define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

<a id='6.3.4'></a>
###7.3.4. Agen 2: SAC

In [46]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.4. Agen SAC

In [47]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return         -0.079505
Cumulative returns    -0.150179
Annual volatility      0.173646
Sharpe ratio          -0.391178
Calmar ratio          -0.288217
Stability              0.010866
Max drawdown          -0.275852
Omega ratio            0.928333
Sortino ratio         -0.534977
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.916725
Daily value at risk   -0.022147
dtype: float64
